In [1]:
import os

import argparse

from models import *  # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *



parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='./cfg/dense_yolov3_4.cfg', help='*.cfg path')
parser.add_argument('--img_path', type=str, default='./data/car/', help='*.img path')
parser.add_argument('--names', type=str, default='./data/visdrone.names', help='*.names path')
parser.add_argument('--weights', type=str, default='./weights/dense_yolov3_4_last_1024.pt', help='weights path')
parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.3, help='object confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.6, help='IOU threshold for NMS')
parser.add_argument('--fourcc', type=str, default='mp4v', help='output video codec (verify ffmpeg support)')
parser.add_argument('--device', default='0', help='device id (i.e. 0 or 0,1) or cpu')
parser.add_argument('--classes', nargs='+', type=int, help='filter by class')
parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
opt = parser.parse_known_args()[0]

device = torch_utils.select_device(device=opt.device)


Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-32GB', total_memory=32480MB)



In [2]:
class model_pred(object):
    def __init__(self, opt=opt, device=device):
        super(model_pred, self).__init__()
        self.opt = opt
        self.device = device

    def load_model(self, weight_path, cfg_path, imgsz):
        model = Darknet(cfg_path, imgsz)
        model.load_state_dict(torch.load(weight_path, map_location=self.device)['model'])
        model.to(self.device).eval()
        return model

    def get_pred(self, img, model):
        with torch.no_grad():
            pred = model(img, augment=False)[0]
            pred = non_max_suppression(pred, self.opt.conf_thres, self.opt.iou_thres,
                                       multi_label=False, classes=self.opt.classes, agnostic=self.opt.agnostic_nms)

        return pred

    def img_process(self, img_dir_update):
        # 对图像的处理
        img0 = cv2.imread(img_dir_update)
        img = letterbox(img0, new_shape=640)[0]
        # Convert
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(self.device)  # to GPU
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        return img, img0


class Inference(model_pred):

    def __init__(self):
        super(Inference, self).__init__()
        self.img_dir = self.opt.img_path
        self.weight_dir = self.opt.weights
        self.cfg_dir = self.opt.cfg

        self.model = self.load_model(self.weight_dir, self.cfg_dir, self.opt.img_size)

    def infer_toresult(self):
        path = self.img_dir
        file_path = os.listdir(path)
        file_path.sort(key = lambda x:int(x[:-4]))
        for img_path in file_path:
            name=img_path[:-4]
            img, img0 = self.img_process(os.path.join(path, img_path))
            t1 = torch_utils.time_synchronized()
            pred = self.get_pred(img, self.model)
            t2 = torch_utils.time_synchronized()
            print('inference time:',t2-t1)


In [3]:
while True:
    infer = Inference()
    infer.infer_toresult()

Model Summary: 254 layers, 6.19432e+07 parameters, 6.19432e+07 gradients
inference time: 0.472398042678833
inference time: 0.014076709747314453
inference time: 0.014065027236938477
inference time: 0.01400446891784668
inference time: 0.01405191421508789
inference time: 0.014070749282836914
inference time: 0.014123678207397461
inference time: 0.013996601104736328
inference time: 0.013956069946289062
inference time: 0.013988018035888672
inference time: 0.014059782028198242
inference time: 0.013944625854492188
inference time: 0.014040708541870117
inference time: 0.014110565185546875
inference time: 0.01398468017578125
inference time: 0.013993978500366211
inference time: 0.014000654220581055
inference time: 0.014089345932006836
inference time: 0.014046669006347656
inference time: 0.01400899887084961
inference time: 0.013972997665405273
inference time: 0.014053821563720703
inference time: 0.014127254486083984
inference time: 0.014037132263183594
inference time: 0.014081954956054688
inference

KeyboardInterrupt: 

In [4]:
import os 
os.getcwd()

'/home/tione/notebook/seu/yolov3-master'